# Training model for bike occupancy for each station. 

### Connect to local database

In [86]:
import mysql.connector
import logging
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np
import pickle
from skimpy import skim

# Configure logging
logging.basicConfig(level=logging.INFO)  # Set logging level to INFO
#its a local database on 127.0.0.1:3306
HOST = '127.0.0.1'
USER = 'root'
PASSWORD = '12345'
DATABASE = 'dublinbikes1104'

connection = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DATABASE
        )
cursor = connection.cursor()
print("Connected to the database")


Connected to the database


### Convert availability data to dataframe

In [87]:
def get_data():
    cursor = connection.cursor()
    cursor.execute(f"SELECT number, timestamp, available_bikes, available_bike_stands FROM dublinbikes1104.availability")
    data = cursor.fetchall()
    select_query = "SELECT COUNT(*) FROM dublinbikes1104.availability;"
    cursor.execute(select_query)

    count = cursor.fetchone()
    print(count)
    cursor.close()
    return data, count

def make_df():
    data, count = get_data()
    print(count)
    df = pd.DataFrame(data, columns=['number','timestamp', 'available_bikes', 'available_bike_stands'])

    # Extract day and hour components
    df['Day'] = pd.to_datetime(df['timestamp'], unit='s').dt.dayofweek
    df['Hour'] = pd.to_datetime(df['timestamp'], unit='s').dt.hour  # Hour of the day

    return df

availability = make_df()
availability.head()

(911294,)
(911294,)


,number,timestamp,available_bikes,available_bike_stands,Day,Hour
0,1,1708510653,12,19,2,10
1,1,1708510801,13,18,2,10
2,1,1708511101,13,18,2,10
3,1,1708511702,13,18,2,10
4,1,1708512301,13,18,2,10


### Convert weather data to dataframe

In [88]:
#write a function that connects to database, takes a station number and creates a df, and adds data from database to df.
def get_weather_data():
    cursor = connection.cursor()
    cursor.execute(f"SELECT timestamp, main, rain, temp, wind_speed FROM dublinbikes1104.weather")
    data = cursor.fetchall()
    cursor.close()
    return data

#create df for weather data
def create_weather_df():
    data = get_weather_data()
    df = pd.DataFrame(data, columns=['timestamp','main', 'rain', 'temp', 'wind_speed'])
    return df

weather = create_weather_df()
weather.head()

    

,timestamp,main,rain,temp,wind_speed
0,1708510653,Clouds,0.0,281.73,6.76
1,1708510654,Clouds,0.0,281.73,6.76
2,1708510801,Clouds,0.0,281.73,6.76
3,1708510802,Clouds,0.0,281.73,6.76
4,1708511101,Clouds,0.0,281.73,6.76


### Merge dataframes by timestamp

In [92]:
def merge_dfs(weather_df, availability_df):
    merged_df = pd.merge(availability_df, weather_df, on='timestamp')
    return merged_df

merged_data = merge_dfs(weather, availability)
merged_data.describe()

,number,timestamp,available_bikes,available_bike_stands,Day,Hour,rain,temp,wind_speed
count,911052.000000,9.110520e+05,911052.000000,911052.000000,911052.000000,911052.000000,911052.000000,911052.000000,911052.000000
mean,57.973513,1.710790e+09,12.053562,19.826844,2.894984,11.786475,0.062567,281.906197,5.631521
std,33.940487,1.237441e+06,9.968163,11.223886,1.927262,6.686984,0.284258,3.350153,2.281611
min,1.000000,1.708511e+09,0.000000,0.000000,0.000000,0.000000,0.000000,273.620000,0.410000
25%,28.000000,1.709888e+09,3.000000,11.000000,1.000000,6.000000,0.000000,279.590000,3.880000
50%,58.000000,1.710869e+09,10.000000,20.000000,3.000000,12.000000,0.000000,281.700000,5.430000
75%,88.000000,1.711833e+09,19.000000,29.000000,5.000000,17.000000,0.000000,283.950000,7.200000
max,117.000000,1.712826e+09,40.000000,40.000000,6.000000,23.000000,7.680000,296.380000,13.430000


In [91]:
import matplotlib.pyplot as plt
missing_weather_timestamps = merged_data[merged_data['rain'].isna()]['timestamp']
missing_timestamps_unique = sorted(missing_weather_timestamps.unique().tolist())  # Get unique timestamps
missing_timestamps_count = len(missing_timestamps_unique)  # Calculate the count of unique timestamps

# Print only the first few elements of the list
print("List of unique timestamps with no weather data (first 10 elements):", missing_timestamps_unique[:10])
print("Count of unique timestamps with no weather data:", missing_timestamps_count)
print("Finished")



import pickle

#Save the list to a file
#with open('missing_timestamps_unique_new.pkl', 'wb') as f:
    #pickle.dump(missing_timestamps_unique, f)




List of unique timestamps with no weather data (first 10 elements): [1709875801, 1709876101, 1709876401, 1709876701, 1709877001, 1709878201, 1709947801, 1712048101, 1712438401]
Count of unique timestamps with no weather data: 9
Finished


In [103]:
print("Before:")
print(merged_data.dtypes)

def clean_df(df):

    df['main'] = df['main'].astype('category')
    df['number'] = df['number'].astype('category')
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df['Day'] = df['Day'].astype('category')
    df['Hour'] = df['Hour'].astype('category')

clean_df(merged_data)
merged_data.dtypes


Before:
number                         category
timestamp                datetime64[ns]
available_bikes                   int64
available_bike_stands             int64
Day                               int32
Hour                              int32
main                           category
rain                            float64
temp                            float64
wind_speed                      float64
dtype: object


number                         category
timestamp                datetime64[ns]
available_bikes                   int64
available_bike_stands             int64
Day                            category
Hour                           category
main                           category
rain                            float64
temp                            float64
wind_speed                      float64
dtype: object

In [104]:
merged_data.head()

,number,timestamp,available_bikes,available_bike_stands,Day,Hour,main,rain,temp,wind_speed
0,1,2024-02-21 10:17:33,12,19,2,10,Clouds,0.0,281.73,6.76
1,1,2024-02-21 10:20:01,13,18,2,10,Clouds,0.0,281.73,6.76
2,1,2024-02-21 10:25:01,13,18,2,10,Clouds,0.0,281.73,6.76
3,1,2024-02-21 10:35:02,13,18,2,10,Clouds,0.0,281.99,6.16
4,1,2024-02-21 10:45:01,13,18,2,10,Clouds,0.0,282.01,6.16


In [95]:
#Print the number of duplicates, without the original rows that were duplicated
print('Number of duplicate (excluding first) rows in the table is: ', merged_data.duplicated().sum())

# Check for duplicate rows. 
# Use "keep=False" to mark all duplicates as true, including the original rows that were duplicated.
print('Number of duplicate rows (including first) in the table is:', merged_data[merged_data.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


In [96]:
# Descriptive statistics for datetime columns
print("Datetime columns descriptive statistics:")
print(merged_data.select_dtypes(include=['datetime64']).describe())

# Descriptive statistics for integer columns
print("\nInteger columns descriptive statistics:")
print(merged_data.select_dtypes(include=['int64']).describe())

# Descriptive statistics for float columns
print("\nFloat columns descriptive statistics:")
print(merged_data.select_dtypes(include=['float64']).describe())

Datetime columns descriptive statistics:
                           timestamp
count                         911052
mean   2024-03-18 19:27:22.646380032
min              2024-02-21 10:17:33
25%              2024-03-08 08:45:02
50%              2024-03-19 17:20:02
75%              2024-03-30 21:10:01
max              2024-04-11 09:00:06

Integer columns descriptive statistics:
       available_bikes  available_bike_stands
count    911052.000000          911052.000000
mean         12.053562              19.826844
std           9.968163              11.223886
min           0.000000               0.000000
25%           3.000000              11.000000
50%          10.000000              20.000000
75%          19.000000              29.000000
max          40.000000              40.000000

Float columns descriptive statistics:
                rain           temp     wind_speed
count  911052.000000  911052.000000  911052.000000
mean        0.062567     281.906197       5.631521
std         0.28

In [105]:
#Printing table of descriptive statistics for categorical features
df_table_categ = merged_data.select_dtypes(['category']).describe().T
df_table_categ

,count,unique,top,freq
number,911052,114,69,9975
Day,911052,7,2,157249
Hour,911052,24,17,44065
main,911052,3,Clouds,717972


In [106]:
category_columns = merged_data.select_dtypes(['category']).columns
for column in category_columns:
    print("--------||--------")
    print(merged_data[column].value_counts(dropna=False))

--------||--------
number
69     9975
33     9490
5      9249
34     9196
9      9169
       ... 
53     7046
116    6995
105    6975
79     6967
104    6854
Name: count, Length: 114, dtype: int64
--------||--------
Day
2    157249
3    142341
1    139194
4    123506
0    118892
5    116478
6    113392
Name: count, dtype: int64
--------||--------
Hour
17    44065
8     43739
16    42644
18    42104
9     41519
7     40652
13    40603
15    39960
12    39920
19    39622
14    39464
10    38860
11    38709
20    37596
6     37104
5     36573
21    36477
22    35335
23    34393
0     33737
4     32898
3     32243
2     31681
1     31154
Name: count, dtype: int64
--------||--------
main
Clouds    717972
Rain      143766
Clear      49314
Name: count, dtype: int64


In [107]:
df_category_mode = pd.DataFrame(index = df_table_categ.index, columns=['mode', 'freq_mode','%mode', '2ndmode', 'freq_2ndmode','%2ndmode'])

for column in category_columns:
  
    df_category_mode.loc[column]['mode'] = merged_data[column].value_counts().keys()[0]
    df_category_mode.loc[column]['freq_mode'] = merged_data[column].value_counts()[df_category_mode.loc[column]['mode']]
    df_category_mode.loc[column]['%mode'] = merged_data[column].value_counts(normalize=True)[df_category_mode.loc[column]['mode']]
 

    if merged_data[column].value_counts().size > 1:
        df_category_mode.loc[column]['2ndmode'] = merged_data[column].value_counts().keys()[1]
        df_category_mode.loc[column]['freq_2ndmode'] = merged_data[column].value_counts()[df_category_mode.loc[column]['2ndmode']]
        df_category_mode.loc[column]['%2ndmode'] = merged_data[column].value_counts(normalize=True)[df_category_mode.loc[column]['2ndmode']]
    else: df_category_mode.loc[column] = '-'

df_category_mode

C:\Users\niall\AppData\Local\Temp\ipykernel_17048\1676522941.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_category_mode.loc[column]['mode'] = merged_data[column].value_counts().keys()[0]
C:\Users\niall\AppData\Local\Temp\ipykernel_

,mode,freq_mode,%mode,2ndmode,freq_2ndmode,%2ndmode
number,69,9975,0.010949,33,9490,0.010417
Day,2,157249,0.172602,3,142341,0.156238
Hour,17,44065,0.048367,8,43739,0.048009
main,Clouds,717972,0.788069,Rain,143766,0.157802


In [108]:
# Prepare %missing column
category_columns_perc_missing  = 100 * (merged_data[category_columns].isnull().sum()/merged_data.shape[0])
#category_columns_perc_missing

# Store the values in a dataframe
df_category_perc_missing = pd.DataFrame(category_columns_perc_missing, columns=['%missing'])
df_category_perc_missing

# Put the columns together to prepare the final table 
df_category_columns_data_quality_report_table = pd.concat([df_table_categ, df_category_mode, df_category_perc_missing], axis=1)

df_category_columns_data_quality_report_table

,count,unique,top,freq,mode,freq_mode,%mode,2ndmode,freq_2ndmode,%2ndmode,%missing
number,911052,114,69,9975,69,9975,0.010949,33,9490,0.010417,0.0
Day,911052,7,2,157249,2,157249,0.172602,3,142341,0.156238,0.0
Hour,911052,24,17,44065,17,44065,0.048367,8,43739,0.048009,0.0
main,911052,3,Clouds,717972,Clouds,717972,0.788069,Rain,143766,0.157802,0.0


In [109]:
df_station_1 = merged_data[merged_data['number'] == 1]
df_station_1.head()

,number,timestamp,available_bikes,available_bike_stands,Day,Hour,main,rain,temp,wind_speed
0,1,2024-02-21 10:17:33,12,19,2,10,Clouds,0.0,281.73,6.76
1,1,2024-02-21 10:20:01,13,18,2,10,Clouds,0.0,281.73,6.76
2,1,2024-02-21 10:25:01,13,18,2,10,Clouds,0.0,281.73,6.76
3,1,2024-02-21 10:35:02,13,18,2,10,Clouds,0.0,281.99,6.16
4,1,2024-02-21 10:45:01,13,18,2,10,Clouds,0.0,282.01,6.16


In [162]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
import numpy as np

def evaluate_regression(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"R^2: {r2}")
    return mae, mse, r2



def run_regression_with_model(df, regressor, model_name):
    X = df[['main', 'rain', 'temp', 'wind_speed', 'Day', 'Hour']]
    y = df['available_bikes']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
    
    numeric_features = ['rain', 'temp', 'wind_speed']
    categorical_features = ['Day', 'Hour', 'main']
    
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])
    
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])
    
    model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', regressor)])
    
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')
    print(f"{model_name} - Cross-validated R^2 scores: {cv_scores}")
    print(f"{model_name} - Average cross-validated R^2 score: {np.mean(cv_scores)}")
    
    model.fit(X_train, y_train)
    
    y_pred_train = model.predict(X_train)
    print(f"\n{model_name} - Training Set Evaluation:\n")
    evaluate_regression(y_train, y_pred_train)
    
    y_pred_test = model.predict(X_test)
    print(f"\n{model_name} - Test Set Evaluation:\n")
    evaluate_regression(y_test, y_pred_test)

    return model

# Replace 'df_station_1' with your actual DataFrame variable
rf_regressor = RandomForestRegressor(n_estimators=30, max_depth=25, random_state=42, n_jobs=-1)


randomForrest = run_regression_with_model(df_station_1, rf_regressor, "Random Forest")




Random Forest - Cross-validated R^2 scores: [-0.4488342   0.04242297  0.24194436 -0.35374569 -0.84912352]
Random Forest - Average cross-validated R^2 score: -0.27346721398663343

Random Forest - Training Set Evaluation:

MAE: 0.7400310440017208
MSE: 1.627045494008087
R^2: 0.9816401714601415

Random Forest - Test Set Evaluation:

MAE: 1.2041793759878068
MSE: 3.9255136563978654
R^2: 0.9564471906167664


### Random Forest shows the best results with the most accurate R^2, the lowest MAE and MSA.

In [155]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np


def predict_available_bikes(day, hour, main_condition):
    """
    Predicts the number of available bikes given the day, hour, and main weather condition.

    Args:
        day (int): The day of the week (0=Monday, 6=Sunday).
        hour (int): The hour of the day (0-23).
        main_condition (str): The main weather condition (e.g., "Clear", "Rain").

    Returns:
        float: The predicted number of available bikes.
    """
    # Create a DataFrame with the input features
    input_df = pd.DataFrame({
        'main': pd.Series([main_condition], dtype='category'), #category
        'rain': [0.00],     # Numeric
        'temp': [279.05],   # Numeric
        'wind_speed': [0.89],  # Numeric
        'Day': [day],         # Numeric or categorical depending on your model
        'Hour': [hour]        # Numeric
        })

    # Use the trained Random Forest model to make a prediction
    # The model pipeline will automatically apply the necessary preprocessing
    predicted_bikes = randomForrest.predict(input_df)[0]  # Get the first prediction

    return predicted_bikes


day = 3  # Thursday
hour = 15  # 3 PM
main_condition = 'Clouds'
predicted_bikes = predict_available_bikes(day, hour, main_condition)
print(f"Predicted available bikes: {round(predicted_bikes)}")

Predicted available bikes: 2


In [160]:
#Show average number of bikes available on thursday at 3pm with clouds

df_station_1[(df_station_1['Day'] == 3) & (df_station_1['Hour'] == 15)&(df_station_1['main'] == 'Clouds')]['available_bikes'].mean()

2.7435897435897436

### Training Random Forrest for every station and saving as pickle files

In [163]:
import os

def save_model_to_pickle(model, filename):
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

# Create the pickle_files folder if it doesn't exist
pickle_files = 'pickle_files_new'
if not os.path.exists(pickle_files):
    os.makedirs(pickle_files)

# Assuming 'merged_data' contains your full dataset
unique_stations = merged_data['number'].unique()
regressor = RandomForestRegressor(n_estimators=30, max_depth=25, random_state=42, n_jobs=-1)

for station_id in unique_stations:
    df_station = merged_data[merged_data['number'] == station_id]
    model_name = f"RandomForest_Station_{station_id}"
    model = run_regression_with_model(df_station, regressor, model_name)
    # Include the folder name in the file path
    filepath = os.path.join(pickle_files, f"{model_name}.pkl")
    save_model_to_pickle(model, filepath)


RandomForest_Station_1 - Cross-validated R^2 scores: [-0.4488342   0.04242297  0.24194436 -0.35374569 -0.84912352]
RandomForest_Station_1 - Average cross-validated R^2 score: -0.27346721398663343

RandomForest_Station_1 - Training Set Evaluation:

MAE: 0.7400310440017209
MSE: 1.627045494008087
R^2: 0.9816401714601415

RandomForest_Station_1 - Test Set Evaluation:

MAE: 1.2041793759878068
MSE: 3.925513656397865
R^2: 0.9564471906167664
RandomForest_Station_2 - Cross-validated R^2 scores: [-0.5870188  -0.0526581  -0.71108576 -0.32550122 -0.01132967]
RandomForest_Station_2 - Average cross-validated R^2 score: -0.3375187104577792

RandomForest_Station_2 - Training Set Evaluation:

MAE: 0.5399468742064487
MSE: 0.7899511615052687
R^2: 0.9644550354428532

RandomForest_Station_2 - Test Set Evaluation:

MAE: 0.9677055953914693
MSE: 2.7492437254352304
R^2: 0.8770708113412303
RandomForest_Station_3 - Cross-validated R^2 scores: [-0.87813305 -1.35553838 -0.22925656 -0.07446575 -0.68992425]
RandomFo